In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"] ="https://dagshub.com/vaidehipawar755/EndToEnd_DSproject.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = 'vaidehipawar755'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '8eea17d116d77cf9c97b571260725d41760dcebd'

In [2]:
import os
%pwd

'/Users/vaidehipawar/Desktop/mlops/EndToEnd_DSproject/research'

In [3]:
os.chdir('../')
%pwd

'/Users/vaidehipawar/Desktop/mlops/EndToEnd_DSproject'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [ ]:

from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)  ## Contains paths and URLs (e.g., from config.yaml)
        self.params = read_yaml(params_filepath)  ## Training hyperparameters (e.g., params.yaml)
        self.schema = read_yaml(schema_filepath)  ## Data schema for validation (e.g., schema.yaml)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig: 
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])


        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name= config.metric_file_name,
            target_column = schema.name,
            mlflow_uri ='https://dagshub.com/vaidehipawar755/EndToEnd_DSproject.mlflow'
        )

        return model_evaluation_config

In [14]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [27]:
from src.datascience.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config:  ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        mse = mean_squared_error(actual, pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            ## saving metrics as local
            scores = {'rmse': rmse, "mae": mae, "r2": r2}
            save_json(path = Path(self.config.metric_file_name), data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('rmse', rmse)
            mlflow.log_metric('r2', r2)
            mlflow.log_metric('mae', mae)

            # model registry does not work with file store

            if tracking_uri_type_store != 'file':

                ## Register the model
                mlflow.sklearn.log_model(model,"model", registered_model_name='ElasticNetModel')
            else:
                mlflow.sklearn.log_model(model,"model")


In [28]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation= ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-04 16:59:14,144: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-04 16:59:14,146: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-04 16:59:14,148: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-04 16:59:14,149: INFO: common: created directory at: artifacts]
[2025-06-04 16:59:14,150: INFO: common: created directory at: artifacts/model_evaluation]
[2025-06-04 16:59:14,314: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/06/04 16:59:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/06/04 16:59:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run peaceful-wren-502 at: https://dagshub.com/vaidehipawar755/EndToEnd_DSproject.mlflow/#/experiments/0/runs/7ccf43350ade4bcd80cfe7d96b337528
🧪 View experiment at: https://dagshub.com/vaidehipawar755/EndToEnd_DSproject.mlflow/#/experiments/0
